# Performing sentiment analysis

## Data Set

`Covid tweets`

The data set is as a result of a scrapping exercise by `GABRIEL PREDA` and was last updated in `2020`

[Download source](https://www.kaggle.com/datasets/gpreda/covid19-tweets)

[DOI](10.34740/kaggle/dsv/1451513)

### Data Content

Tweets with #covid19 hashtag. Collection started on 25/7/2020, with an initial 17k batch and will continue on a daily basis.

The data is in the form of `csv`



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# data set

# data path
path = "drive/My Drive/covid19_tweets.csv"

# reading the data
covidData = pd.read_csv(path)
covidData.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


# Analysis

Using python natural language toolkit `NLTK`

## Initial analysis

### Vader Model

Uses `a bag of words approach`

This is a representation of text illustrating the occurrence of words within a document

It usually contains two parts

1. Vocabulary - Known words
2. Measure - presence of known words

Refereed to as a bag of words because `structure and order of words is discarded`

The analysis uses a `histogram` of the occurrence of each known word in a sentence


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# using ggplot style plots
plt.style.use('ggplot')

# natural language toolkit
import nltk

In [ ]:
# type of data being used
covidData['text'].values[0]

'If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0'

In [ ]:
covidData.shape

(179108, 13)

## Description

there are a total of `179108` tweets with `13 columns`

The key columns are the `text` and the `users_name` columns

The text column reprersents the tweets posted.The following is an example

```
If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0
```

# Analysis

## Pretrained Model Comparison

### Roberta Model

sourced from `hugging face` 🤗

### Huggingface Pipeline

